In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from dotenv import load_dotenv
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd
import re
import csv
import json
import os
from urllib.parse import quote
from datetime import datetime, timedelta
import traceback
import random
from bs4 import BeautifulSoup as bs



In [2]:
# Fonction pour simuler un comportement humain avec des pauses aléatoires afin que linkedin ne detecte pas
def random_pause(min_time=2, max_time=5):
    time.sleep(random.uniform(min_time, max_time))


# Simule un clic sur une zone aléatoire pour "flouter"
def simulate_random_click(driver):
    try:
        elements = driver.find_elements(By.TAG_NAME, "a")
        if elements:
            random.choice(elements).click()
            print("Clic aléatoire simulé.")
            random_pause(3, 5)
    except Exception as e:
        print(f"Aucune zone trouvée pour clic aléatoire : {e}")

def save_cookies(driver, file_path):
    cookies = driver.get_cookies()
    print(f"Cookies récupérés : {cookies}")  
    with open(file_path, "w") as file:
        json.dump(cookies, file)
    print(f"Cookies sauvegardés dans {file_path}")


def load_cookies(driver, file_path):
    with open(file_path, "r") as file:
        cookies = json.load(file)
    for cookie in cookies:
        driver.add_cookie(cookie)


In [3]:
# Chargement desvariables d'environnement
load_dotenv()

LINKEDIN_EMAIL = os.getenv("LINKEDIN_EMAIL")
LINKEDIN_PASSWORD = os.getenv("LINKEDIN_PASSWORD")

if not LINKEDIN_EMAIL or not LINKEDIN_PASSWORD:
    raise ValueError("Les identifiants LinkedIn sont manquants dans le fichier .env")


In [4]:
# Configuration dans le navigateur Chrome
chrome_options = Options()
chrome_options.add_argument("--disable-dev-shm-usage")
options = Options()
options.add_argument("--headless")  
chrome_options.add_experimental_option('excludeSwitches', ['enable-automation'])
# chrome_options.add_experimental_option('useAutomationExtension', False)
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.5672.126 Safari/537.36")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

In [5]:
def login_with_cookies(driver, cookie_file):
    # Ouvrir la page de connexion
    driver.get("https://www.linkedin.com/uas/login")

# Attendre que l'utilisateur se connecte manuellement
    input("Appuyez sur Entrée après vous être connecté...")

# Enregistrement des cookies
    cookies = driver.get_cookies()
    with open("cookies.json", "w") as file:
        json.dump(cookies, file)

    print("Cookies enregistrés.")
    with open("cookies.json", "r") as file:
        cookies = json.load(file)
        for cookie in cookies:
            driver.add_cookie(cookie)

    driver.refresh()

# Vérification de la connexion
    try:
        profile_icon = driver.find_element(By.CSS_SELECTOR, "img.feed-identity-module__member-photo")
        print("Connexion réussie.")
    except:
        print("Non connecté.")

In [6]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random

def random_pause(min_seconds, max_seconds):
    time.sleep(random.uniform(min_seconds, max_seconds))

def search_keyword(driver, keyword):
    try:
        print(f"Recherche pour le mot-clé : {keyword}")
        search_input = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "search-global-typeahead__input"))
        )
        search_input.clear()
        search_input.send_keys(keyword)
        search_input.send_keys(Keys.RETURN)
        random_pause(5, 10)
    except Exception as e:
        print(f"Erreur lors de la recherche pour le mot-clé '{keyword}': {e}")

def scrape_manual_results(driver, keyword, csv_writer):
    try:
        print(f"Extraction des résultats pour : {keyword}")
        posts = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "fie-impression-container"))
        )
        print(f"{len(posts)} posts trouvés pour le mot-clé '{keyword}'.")

        for post in posts:
            try:
                likes = post.find_element(By.CLASS_NAME, "social-details-social-counts__reactions-count").text
                comments = post.find_element(By.CLASS_NAME, "social-details-social-counts__count-value").text
                republications = post.find_element(By.CLASS_NAME, "social-details-social-counts__count-value-hover").text
                author = post.find_element(By.CLASS_NAME, "update-components-actor__title").text
                author_description = post.find_element(By.CLASS_NAME, "update-components-actor__description").text
                time_posted = post.find_element(By.CLASS_NAME, "update-components-actor__sub-description").text

                # Sauvegarde des données dans le CSV
                csv_writer.writerow([keyword, likes, comments, republications, author, author_description, time_posted])

                random_pause(1, 3)
            except Exception as e:
                print(f"Erreur lors de l'extraction d'un post : {e}")

    except Exception as e:
        print(f"Erreur lors du scraping des résultats pour '{keyword}': {e}")

if __name__ == "__main__":
    try:
        # Lancement du driver Selenium
        cookie_file = "cookies.json"
        driver.get("https://www.linkedin.com/feed/")
        print("Accès à LinkedIn réussi.")

        # Création ou ouverture d'un fichier CSV
        with open("linkedin_posts.csv", mode="w", newline="", encoding="utf-8") as csv_file:
            csv_writer = csv.writer(csv_file)
            
            # En-têtes des colonnes
            csv_writer.writerow(["Mot-clé", "Likes", "Commentaires", "Republications", "Auteur", "Description Auteur", "Temps Posté"])

            # Saisir les mots-clés dynamiquement
            while True:
                keyword = input("Entrez un mot-clé pour la recherche (ou appuyez sur 'Entrée' pour terminer) : ").strip()
                if not keyword:
                    print("Fin de la session de recherche.")
                    break

                search_keyword(driver, keyword)
                scrape_manual_results(driver, keyword, csv_writer)
                random_pause(10, 20)

    finally:
        driver.quit()
        print("Session terminée.")


Accès à LinkedIn réussi.
Fin de la session de recherche.
Session terminée.
